Programmieren 3 - Parallelisierung

Peter Rösch, Fakultät für Informatik

Hochschule Augsburg, 2017/2018

# TSP - Implementierung

Die Implementierung des TSP-Verfahrens steht als Modul zur Verfügung.

In [ ]:
import TSP
%matplotlib inline

In [ ]:
start_pos, nr_of_cities = 0, 9
end_pos = start_pos + nr_of_cities

result = TSP.shortest_closed_path(((TSP.staedte_positionen[start_pos], ), 
                            TSP.staedte_positionen[start_pos+1:end_pos]))
TSP.plot_closed_path(result[1])
print('Länge:', result[0])

# Parallelisierung - Einführung

Einen Artikel von H. Sutter mit dem Titel *The Free Lunch Is Over: A Fundamental Turn Toward Concurrency in Software* finden Sie [hier](http://www.gotw.ca/publications/concurrency-ddj.htm).

Nach *Amdahls Gesetz* können in Sie in den [Safari-Books](http://proquest.tech.safaribooksonline.de) suchen oder im entsprechenden [Wikipedia-Artikel](http://de.wikipedia.org/wiki/Amdahlsches_Gesetz).

# Threads und Prozesse

* Was hat [dieses Bild](https://www.schick-seitenblicke.at/en/wp-content/uploads/sites/2/2012/11/MG_1422_kompr.jpg) mit Threads und Prozessen zu tun? Ein eher technisches Bild finden Sie [hier](http://www.fmc-modeling.org/category/projects/apache/amp/images/11-OS_Concepts/processes+threads_BD.gif).

* Welche Auswirkungen kaben die Unterschiede zwischen Threads und Prozessen auf die Programmierung?

* Was bedeutet die Abkürzung [GIL](https://wiki.python.org/moin/GlobalInterpreterLock)?

* Haben Sie schon einmal mit [Posix Threads](https://computing.llnl.gov/tutorials/pthreads) gearbeitet?

Im Archiv [pi_monte_carlo_ANSI_C.tar.bz2](https://moodle.hs-augsburg.de/pluginfile.php/47010/mod_folder/content/0/pi_monte_carlo_ANSI_C.tar.bz2?forcedownload=1) (Moodle) finden Sie eine Implementierung der Monte Carlo-Simulation in ANSI C, die POSIX-Threads zur Parallelisierung einsetzt. Zum Vergleich gibt es eine nicht parallelisierte Version unter *monteCarloUnitSphereSingle*. Die Programme erstellen Sie wie folgt:

    cd monteCarloUnitSphereSingle
    gcc -O3 -o monteCarloUnitSphereSingle monteCarloUnitSphereSingle.c -pthread -lm
    time ./monteCarloUnitSphereSingle 10 10
    cd monteCarloUnitSphereMulti
    gcc -O3 -o monteCarloUnitSphereMulti monteCarloUnitSphereMulti.c -pthread -lm
    time ./monteCarloUnitSphereMulti 10 10 4
    
Das letzte Argument beim Aufruf des Programms monteCarloUnitSphereMulti bezeichnet die Anzahl der Threads, die gestartet werden (in diesem Fall 4 Threads).

In einem Python-Programm können Prozesse, die eine bestimmte Funktion ausführen, direkt erzeugt und gestartet werden. Dazu verwenden wir das sehr mächtige Paket [multiprocessing](https://docs.python.org/3.6/library/multiprocessing.html).

In [ ]:
import multiprocessing, os
from time import sleep

def function(name):
    sleep(1)
    print('pid:', os.getpid())
    print('hello', name)
    
# Die folgende Zeile ist unter Windows unbedingt notwendig!
if __name__ == '__main__':
    print('main pid:', os.getpid())
    p1 = multiprocessing.Process(target=function, args=('p1',))
    p2 = multiprocessing.Process(target=function, args=('p2',))
    p1.start()
    p2.start()
    p1.join()
    p2.join()
    print('done')

Prozesse können über *pipes* miteinander kommunizieren.

In [ ]:
import multiprocessing

def f(p):
    while True:
        message = p.recv()
        if message[0] == 'getLost':
            p.send('finishing'); break
        else:
            p.send([message, 'received!'])

if __name__ == '__main__':
    myEnd, otherEnd = multiprocessing.Pipe()
    process = multiprocessing.Process(target=f, args=(otherEnd,))
    process.start()
    for arg in ((1,2,3), ('hallo',), (3, 'test'), ('getLost',)):
        myEnd.send(arg)
        print(myEnd.recv())

# Das Modul multiprocessing - Details

Die Anzahl der Prozesse, die gestartet wird, sollte von der Anzahl der vorhandenen CPU-Kerne abhängen:

In [ ]:
import multiprocessing
import math
nrOfCores = multiprocessing.cpu_count()
print('nrOfCores:', nrOfCores)

Was hat [dieses Bild](https://colourbox.com/preview/2836733-pile-of-pancakes-with-an-oil-slice-on-a-round-plate.jpg) mit der parallelisierten Abarbeitung von Aufgaben zu tun?

Das folgende Beispiel zeigt die parallelisierte Berechnung von Quadratzahlen.

In [ ]:
def f(qIn, qOut):
    while True:
        x = qIn.get()
        result = x*x
        qOut.put(result)
        qIn.task_done()

if __name__ == '__main__':
    argumentQueue = multiprocessing.JoinableQueue()
    resultQueue = multiprocessing.Queue()
    nrOfProcesses = multiprocessing.cpu_count()
    processes = [multiprocessing.Process(
                            target = f,
                            args = (argumentQueue, resultQueue))
                    for i in range(nrOfProcesses)]
    for i in range(0, 10):
        argumentQueue.put(i)
    for p in processes:
        p.start()  
    argumentQueue.join()
    for p in processes:
        p.terminate()
    for i in range(10):
        print(resultQueue.get(), end=' ')

Wie können wir den TSP-Algorithmus nach diesem Muster parallelisieren?

Um den TSP so zu parallelisieren, muss zunächst eine Liste mit Argumenten erstellt werden, die die Teilaufgaben definiert:

In [ ]:
def create_argument_list(path_completed, path_ahead, r_depth, l):
    if len(path_completed) == r_depth:
        l.append((path_completed, path_ahead))
    else:
        for i in range(len(path_ahead)):
            create_argument_list(path_completed + (path_ahead[i],) , 
                                path_ahead[:i] + path_ahead[i+1:],
                                r_depth, l)

Vorbereitungen für die parallelisierte Berechnung des Rundwegs durch *nr_of_cities* Städte:

In [ ]:
nr_of_cities = 10
r_depth = 2
l = []
create_argument_list((TSP.staedte_positionen[0], ), 
            TSP.staedte_positionen[1:nr_of_cities], r_depth, l)

In [ ]:
l[:2]

Die Worker-Funktion:

In [ ]:
import TSP
def worker_TSP(q_in, q_out):
    while True:
        arguments = q_in.get()
        result = TSP.shortest_closed_path(arguments)
        q_out.put(result)
        q_in.task_done()

Berechnung unter Verwendung von *Queues*:

In [ ]:
%%timeit
in_queue = multiprocessing.JoinableQueue()
result_queue = multiprocessing.Queue()

processes = []
for i in range(nrOfCores):
    p = multiprocessing.Process(target = worker_TSP, 
                                args = (in_queue, result_queue))
    processes.append(p)
    p.start()
    
for parameter_set in l:
    in_queue.put(parameter_set)
    
import time

in_queue.join()

result_list = []
while not result_queue.empty():
    result_list.append(result_queue.get())

min_path = min(result_list)

for p in processes:
    p.terminate()

Was vermuten Sie hinter dem Begriff [Worker Pool](http://www.slate.com/content/dam/slate/blogs/quora/2016/07/09/is_it_better_to_be_a_worker_bee_or_a_killer_bee_on_the_job/51341806-top-view-of-worker-bees-that-were-breed-by-self-taught.jpg.CROP.promo-xlarge2.jpg)?

Wieso ist der Befehl *map* für die Parallelisierung auf Prozess-Ebene sehr hilfreich?

Nicht-parallele Version:

In [ ]:
%%timeit
l2 = map(TSP.shortest_closed_path, l)
resultSerial = min(l2)

Parallelisierte Version:

In [ ]:
%%timeit
process_pool = multiprocessing.Pool(processes = nrOfCores)
l2 = process_pool.map(TSP.shortest_closed_path, l)
result_parallel = min(l2)
process_pool.close()

# Parallelisierung mit cython

Das Modul [cython.parallel](http://docs.cython.org/en/stable/src/userguide/parallelism.html) erlaubt es, Cython-Code auf Thread-Ebene unter Verwendung von [OpenMP](http://www.openmp.org) zu parallelisieren.

Erzeugung eines *numpy*-Arrays mit normalverteilten Zufallszahlen:

In [ ]:
import numpy
nrOfListEntries = 10**6
a = numpy.random.normal(3, 1, size = nrOfListEntries).astype(numpy.float32)

Berechnung von Mittelwert und Standardabweichung in Python:

In [ ]:
import numpy
from math import sqrt

def python_stat(l):
    """
    calculate mean and standard deviation of data stored in a list
    using pure python functions.
    Args:
        l list containing numbers
    Returns:
       (mean, standardDeviation) tuple
    """
    accumulator = 0.0
    N = len(l)
    for x in l:
        accumulator += x
    average = accumulator / N
    accumulator = 0.0
    for x in l:
        tmp = x - average
        accumulator += tmp * tmp
    standard_deviation = sqrt(accumulator / (N - 1))
    return (average, standard_deviation)

In [ ]:
%timeit python_stat(a)
print(python_stat(a))

Jetzt mit *cython* und *numpy*, siehe http://www.numpy.org.

In [ ]:
%load_ext Cython

In [ ]:
%%cython 

"""
Implementation of mean and standard deviation calculation using
cython
"""
#from __future__ import with_statement
import numpy
cimport numpy
cimport cython
from libc.math cimport sqrt

ctypedef numpy.float32_t dtype_t

@cython.boundscheck(False)
@cython.cdivision(True)
cpdef numpy.ndarray[dtype_t, ndim=1] cython_stat(
        numpy.ndarray[dtype_t, ndim=1] l):
    """
    calculate mean and standard deviation of data stored in a list
    using cython parallel.
    Args:
        l numpy array containing numbers
    Returns:
        list [mean, standardDeviation] 
    """
    cdef double average, standard_deviation, tmp
    cdef double accumulator = 0.0
    cdef long N, i

    N = len(l)
    for i in range(N):
        accumulator += l[i]
    average = accumulator / N
    accumulator = 0.0
    for i in range(N):
        tmp = l[i] - average
        accumulator += tmp * tmp
    standard_deviation = sqrt(accumulator / (N - 1))

    result = numpy.array((average, standard_deviation), numpy.float32)
    return result

In [ ]:
%timeit cython_stat(a)
print(cython_stat(a))

Eine kleine Änderung und das Programm nutzt Threads zur Parallelisierung:

In [ ]:
%%cython -f -c-fopenmp --link-args=-fopenmp

"""
Implementation of mean and standard deviation calculation using
cython.parallel
"""
#from __future__ import with_statement

from cython.parallel import prange
import numpy
cimport numpy
cimport cython
from libc.math cimport sqrt

ctypedef numpy.float32_t dtype_t

@cython.boundscheck(False)
@cython.cdivision(True)
cpdef numpy.ndarray[dtype_t, ndim=1] cython_stat_parallel(
        numpy.ndarray[dtype_t, ndim=1] l):
    """
    calculate mean and standard deviation of data stored in a list
    using cython parallel.
    Args:
        l numpy array containing numbers
    Returns:
        list [mean, standardDeviation] 
    """
    cdef double average, standard_deviation, tmp
    cdef double accumulator = 0.0
    cdef long N, i

    N = len(l)
    for i in prange(N, nogil=True):
        accumulator += l[i]
    average = accumulator / N
    accumulator = 0.0
    for i in prange(N, nogil=True):
        tmp = l[i] - average
        accumulator += tmp * tmp
    standard_deviation = sqrt(accumulator / (N - 1))

    result = numpy.array((average, standard_deviation), numpy.float32)
    return result

In [ ]:
%timeit cython_stat_parallel(a)
print(cython_stat_parallel(a))

# Aufgaben, Abgabe bis 14.12.2017

## Einführung

1. Finden Sie heraus, welche Studierenden in Ihrem Team bereits parallelisierte Anwendungen eingesetzt haben und wie diese Anwendungen heißen.
1. Warum werden heute nicht alle Anwendungen grundsätzlich so programmiert, dass sie mehrer Prozessor-Kerne unterstützen?
1. Moderne Grafikkarten besitzen tausende von leistungsfähigen Rechen-Einheiten. Warum stattet man Rechner immer noch mit Prozessoren aus, statt die Grafikkarten für alle Anwendungen einzusetzen?
1. Welche Beschleunigung durch Parallelisierung erwarten Sie sich für die TSP-Implementierung? (Schätzung mit Begründung)

## Threads und Prozesse

1. **Freiwillig:** Schauen Sie sich die gegebenen C-Programme genauer an und finden Sie heraus, wie die Synchronisation stattfindet. Eine Recherche in Fachbüchern oder dem Internet kann dabei hilfreich sein.
1. **Freiwillig:** Compilieren und starten Sie die C-Programme und analysieren Sie die Ausgabe des Programms auf einem der Rechner im Labor M3.03. Was bedeuten die Begriffe *real, user* und *sys* und wie erklären Sie sich die Unterschiede zwischen den verschiedenen Implementierung bezüglich der Ausgaben von *time*?
1. Können Sie sich bei dem Beispiel zur Erzeugung von Prozessen darauf verlassen, dass "hello p1" *vor* "hello p2" ausgegeben wird? (Begründung)
1. Beschreiben Sie (Text, Skizze), wie man die rekursive TSP-Implementierung unter Verwendung von Prozessen und *Pipes* parallelisieren könnte. Wäre diese Implementierung ihrer Meinung nach schlechter oder besser zu verstehen als die ANSI-C-Programme?

## Das Module multiprocessing

1. Arbeiten Sie die vorgegebenen Beispiele nochmals durch und stellen Sie ggf. Fragen an den Dozenten.
1. Starten Sie die parallelisierten TSP-Berechnungen auf einem Rechner im Labor M2.02 und vergleichen Sie die Rechenzeiten mit der der nicht parallelisierten Version.
1. Warum wird die Aufgabe nicht in genau so viele Teilaufgaben unterteilt wie man CPU-Kerne hat?
1. Was bewirkt der Befehl *inQueue.join()*?
1. Erstellen Sie zwei parallelisierte Implementierung ihrer $\pi$-Bistimmung nach Monte Carlo, wovon eine auf *map* und eine auf *Queues* basiert. Vergleichen Sie die Rechenzeiten mit denen ihrer bisherigen Version. Wurden Ihre Erwartungen bezüglich der Beschleunigung erfüllt?

## Parallelisierung mit cython

1. Was leistet OpenMP (siehe [hier](http://www.openmp.org/resources/tutorials-articles))?
1. Welche Unterschiede gibt es zwischen der Parallelisierung mit *multiprocessing* und der Parallelisierung mit *cython* und *OpenMP*?
1. Analysieren Sie die oben angegebene Beispiele und stellen Sie ggf. Fragen an den Dozenten.
1. Ergänzen Sie die vorgegebenen Beispiele so, dass auch die Schiefe (skewness) und die Wölbung (kurtosis) einer Menge von Zufallszahlen, die einem *numpy*-Array gespeichert sind, mit *cython* berechnet werden. Welche Beschleunigung durch Parallelisierung konnten Sie erreichen?

# Überprüfung

1. Was bedeutet *Synchronisation* im Zusammenhang mit der Parallelisierung von Computerprogrammen? (max. vier Sätze)
1. Erklären Sie die wesentlichen Unterschiede zwischen Prozessen und Threads. (max. vier Sätze)
1. Nennen Sie zwei wichtige Gründe, warum die parallelisierte Version eines Programms auf einem Rechner mit N Prozessor-Kernen in der Regel nicht um einen Faktor N schneller läuft. (ca. zwei Sätze)